In [ ]:
!pip -q install -U ultralytics opencv-python numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 72.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.1 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.1 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.1 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.1 which is incompatible.


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import files
from pathlib import Path

def apply_terminator_grade(frame_bgr, t, add_edges=True, add_scanlines=True):
    img = frame_bgr.copy()
    h, w = img.shape[:2]

    b = img[:, :, 0].astype(np.float32) * 0.25
    g = img[:, :, 1].astype(np.float32) * 0.45
    r = img[:, :, 2].astype(np.float32) * 1.25 + 15.0
    img = np.clip(np.stack([b, g, r], axis=2), 0, 255).astype(np.uint8)

    if add_edges:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 80, 160)
        edges_bgr = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        img = cv2.addWeighted(img, 1.0, edges_bgr, 0.35, 0)

    if add_scanlines:
        spacing = 4
        img[::spacing, :, :] = (img[::spacing, :, :] * 0.55).astype(np.uint8)

        bar_y = int((t * 120) % max(1, h))
        y1, y2 = max(0, bar_y - 8), min(h, bar_y + 8)
        if y2 > y1:
            bar = img[y1:y2].astype(np.float32)
            bar[:, :, 2] = np.clip(bar[:, :, 2] * 1.35 + 20, 0, 255)
            img[y1:y2] = bar.astype(np.uint8)

    noise = np.random.randint(-8, 9, img.shape, dtype=np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    yy, xx = np.mgrid[0:h, 0:w]
    cx, cy = w / 2.0, h / 2.0
    dist = np.sqrt(((xx - cx) / cx) ** 2 + ((yy - cy) / cy) ** 2)
    vign = np.clip(1.0 - 0.35 * dist, 0.65, 1.0).astype(np.float32)
    img = (img.astype(np.float32) * vign[:, :, None]).astype(np.uint8)
    return img

def draw_hud(img, dets, target, fps, frame_idx):
    out = img
    h, w = out.shape[:2]

    for x in range(0, w, 80):
        cv2.line(out, (x, 0), (x, h), (0, 0, 50), 1)
    for y in range(0, h, 60):
        cv2.line(out, (0, y), (w, y), (0, 0, 50), 1)

    cv2.putText(out, "CYBERDYNE VISION SYSTEM", (15, 28),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 230), 2, cv2.LINE_AA)
    cv2.putText(out, f"FPS:{fps:5.1f}  FRAME:{frame_idx}", (15, 54),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 200), 2, cv2.LINE_AA)

    for d in dets:
        (x1, y1, x2, y2) = d["xyxy"]
        is_target = target is not None and d is target

        color = (0, 0, 255) if is_target else (0, 0, 140)
        thickness = 3 if is_target else 1
        cv2.rectangle(out, (x1, y1), (x2, y2), color, thickness)

        tag = f'{d["cls_name"].upper()} {d["conf"]:.2f}'
        if is_target:
            tag += "  [TERMINATE]"

        (tw, th), _ = cv2.getTextSize(tag, cv2.FONT_HERSHEY_SIMPLEX, 0.55, 2)
        y_text = max(0, y1 - 8)
        cv2.rectangle(out, (x1, y_text - th - 6), (x1 + tw + 6, y_text + 4), (0, 0, 0), -1)
        cv2.putText(out, tag, (x1 + 3, y_text),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 0, 230), 2, cv2.LINE_AA)

    if target is not None:
        x1, y1, x2, y2 = target["xyxy"]
        cx, cy = (x1 + x2)//2, (y1 + y2)//2
        size = 18
        cv2.line(out, (cx - size, cy), (cx + size, cy), (0, 0, 255), 2)
        cv2.line(out, (cx, cy - size), (cx, cy + size), (0, 0, 255), 2)
        cv2.circle(out, (cx, cy), 28, (0, 0, 255), 2)

        status = f'TARGET:{target["cls_name"].upper()}  CONF:{target["conf"]:.2f}'
        cv2.putText(out, status, (15, h - 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 230), 2, cv2.LINE_AA)
        cv2.putText(out, "STATUS: LOCKED", (15, h - 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 230), 2, cv2.LINE_AA)
    else:
        cv2.putText(out, "STATUS: SEARCHING", (15, h - 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 200), 2, cv2.LINE_AA)

    return out

def run_video(in_path, out_path="out_terminator.mp4", model_name="yolov8n.pt",
              target_class="person", conf=0.25, iou=0.7, imgsz=640, max_frames=None):
    model = YOLO(model_name)

    cap = cv2.VideoCapture(str(in_path))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open: {in_path}")

    fps_in = cap.get(cv2.CAP_PROP_FPS)
    if not fps_in or fps_in <= 0:
        fps_in = 30.0

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(out_path, fourcc, fps_in, (w, h))

    frame_idx = 0
    t = 0.0
    fps_est = 0.0
    prev = cv2.getTickCount()

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        # YOLO
        res = model.predict(source=frame, conf=conf, iou=iou, imgsz=imgsz, verbose=False)[0]
        dets = []
        if res.boxes is not None and len(res.boxes) > 0:
            xyxy = res.boxes.xyxy.cpu().numpy()
            confs = res.boxes.conf.cpu().numpy()
            clss = res.boxes.cls.cpu().numpy().astype(int)
            names = res.names
            for (x1, y1, x2, y2), c, cid in zip(xyxy, confs, clss):
                dets.append({
                    "xyxy": (int(x1), int(y1), int(x2), int(y2)),
                    "conf": float(c),
                    "cls_id": int(cid),
                    "cls_name": (names[cid] if isinstance(names, (list, dict)) else str(cid)) if names is not None else str(cid)
                })

        # pick target
        tc = target_class.strip().lower()
        candidates = [d for d in dets if d["cls_name"].lower() == tc]
        target = None
        if candidates:
            target = max(candidates, key=lambda d: ( (d["xyxy"][2]-d["xyxy"][0])*(d["xyxy"][3]-d["xyxy"][1]) * d["conf"] ))
        elif dets:
            target = max(dets, key=lambda d: d["conf"])

        # fps estimate
        now = cv2.getTickCount()
        dt = (now - prev) / cv2.getTickFrequency()
        prev = now
        if dt > 0:
            fps_est = 0.9 * fps_est + 0.1 * (1.0/dt) if fps_est > 0 else (1.0/dt)

        # effect + HUD
        t += 1.0 / fps_in
        stylized = apply_terminator_grade(frame, t)
        hud = draw_hud(stylized, dets, target, fps_est, frame_idx)

        writer.write(hud)
        frame_idx += 1
        if max_frames is not None and frame_idx >= max_frames:
            break

    cap.release()
    writer.release()
    return out_path


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
uploaded = files.upload()  # mp4等を選択
in_file = next(iter(uploaded.keys()))
out_file = run_video(in_file, out_path="out_terminator.mp4", target_class="person", model_name="yolov8n.pt")
out_file



Saving 36510-411342239_small.mp4 to 36510-411342239_small.mp4


'out_terminator.mp4'

# Version2

In [1]:
!pip -q install -U ultralytics opencv-python numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 36.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.1 which is incompatible.
prophet 1.2.2 requires numpy<2.4.0,>=1.15.4, but you have numpy 2.4.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.1 which is incompatible.


In [2]:
!apt-get -qq update
!apt-get -qq install -y ffmpeg


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
from google.colab import files

print("① 入力動画（mp4）をアップロードしてください（例: input.mp4）")
uploaded1 = files.upload()

print("\n② overlay動画（t800-vision の overlay1.mp4 / overlay2.mp4 など）をアップロードしてください")
uploaded2 = files.upload()

# 任意：音声も入れるなら
print("\n（任意）③ overlay音声（overlay1.mp3 など）をアップロードする場合は選択してください（スキップ可）")
uploaded3 = files.upload()

① 入力動画（mp4）をアップロードしてください（例: input.mp4）


Saving sample_small.mp4 to sample_small.mp4

② overlay動画（t800-vision の overlay1.mp4 / overlay2.mp4 など）をアップロードしてください


Saving overlay1.mp4 to overlay1.mp4

（任意）③ overlay音声（overlay1.mp3 など）をアップロードする場合は選択してください（スキップ可）


Saving overlay1.mp3 to overlay1.mp3


In [4]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
from pathlib import Path
import os

def apply_terminator_grade(frame_bgr: np.ndarray, t: float,
                           add_edges: bool = True, add_scanlines: bool = True) -> np.ndarray:
    img = frame_bgr.copy()
    h, w = img.shape[:2]

    b = img[:, :, 0].astype(np.float32)
    g = img[:, :, 1].astype(np.float32)
    r = img[:, :, 2].astype(np.float32)

    b *= 0.25
    g *= 0.45
    r = r * 1.25 + 15.0

    img = np.stack([b, g, r], axis=2)
    img = np.clip(img, 0, 255).astype(np.uint8)

    if add_edges:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 80, 160)
        edges_bgr = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        img = cv2.addWeighted(img, 1.0, edges_bgr, 0.35, 0)

    if add_scanlines:
        spacing = 4
        img[::spacing, :, :] = (img[::spacing, :, :] * 0.55).astype(np.uint8)
        bar_y = int((t * 120) % max(1, h))
        y1 = max(0, bar_y - 8)
        y2 = min(h, bar_y + 8)
        if y2 > y1:
            bar = img[y1:y2, :, :].astype(np.float32)
            bar[:, :, 2] = np.clip(bar[:, :, 2] * 1.35 + 20, 0, 255)
            img[y1:y2, :, :] = bar.astype(np.uint8)

    noise = np.random.randint(-8, 9, img.shape, dtype=np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    yy, xx = np.mgrid[0:h, 0:w]
    cx, cy = w / 2.0, h / 2.0
    dist = np.sqrt(((xx - cx) / cx) ** 2 + ((yy - cy) / cy) ** 2)
    vign = np.clip(1.0 - 0.35 * dist, 0.65, 1.0).astype(np.float32)
    img = (img.astype(np.float32) * vign[:, :, None]).astype(np.uint8)
    return img


def draw_hud(img: np.ndarray, dets, target, fps: float, frame_idx: int) -> np.ndarray:
    out = img
    h, w = out.shape[:2]

    # grid
    for x in range(0, w, 80):
        cv2.line(out, (x, 0), (x, h), (0, 0, 50), 1)
    for y in range(0, h, 60):
        cv2.line(out, (0, y), (w, y), (0, 0, 50), 1)

    # header
    cv2.putText(out, "CYBERDYNE VISION SYSTEM", (15, 28),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 230), 2, cv2.LINE_AA)
    cv2.putText(out, f"FPS:{fps:5.1f}  FRAME:{frame_idx}", (15, 54),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 200), 2, cv2.LINE_AA)

    # detections
    for d in dets:
        x1, y1, x2, y2 = d["xyxy"]
        is_target = (target is not None) and (d is target)

        color = (0, 0, 255) if is_target else (0, 0, 140)
        thickness = 3 if is_target else 1
        cv2.rectangle(out, (x1, y1), (x2, y2), color, thickness)

        tag = f'{d["cls_name"].upper()} {d["conf"]:.2f}'
        if is_target:
            tag += "  [TERMINATE]"

        (tw, th), _ = cv2.getTextSize(tag, cv2.FONT_HERSHEY_SIMPLEX, 0.55, 2)
        y_text = max(0, y1 - 8)
        cv2.rectangle(out, (x1, y_text - th - 6), (x1 + tw + 6, y_text + 4), (0, 0, 0), -1)
        cv2.putText(out, tag, (x1 + 3, y_text),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 0, 230), 2, cv2.LINE_AA)

    # target crosshair
    if target is not None:
        x1, y1, x2, y2 = target["xyxy"]
        cx, cy = (x1 + x2)//2, (y1 + y2)//2
        size = 18
        cv2.line(out, (cx - size, cy), (cx + size, cy), (0, 0, 255), 2)
        cv2.line(out, (cx, cy - size), (cx, cy + size), (0, 0, 255), 2)
        cv2.circle(out, (cx, cy), 28, (0, 0, 255), 2)

        status = f'TARGET:{target["cls_name"].upper()}  CONF:{target["conf"]:.2f}'
        cv2.putText(out, status, (15, h - 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 230), 2, cv2.LINE_AA)
        cv2.putText(out, "STATUS: LOCKED", (15, h - 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 230), 2, cv2.LINE_AA)
    else:
        cv2.putText(out, "STATUS: SEARCHING", (15, h - 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 200), 2, cv2.LINE_AA)

    return out


class OverlayPlayer:
    """
    overlay動画の輝度からマスクを作り、screenブレンドで合成。
    黒背景（暗部）ほど透明扱い。
    """
    def __init__(self, overlay_path: str, alpha: float = 0.85, threshold: int = 10):
        self.overlay_path = overlay_path
        self.alpha = float(alpha)
        self.threshold = int(threshold)
        self.cap = cv2.VideoCapture(overlay_path)
        if not self.cap.isOpened():
            raise RuntimeError(f"Cannot open overlay video: {overlay_path}")

    def read(self) -> np.ndarray:
        ok, frame = self.cap.read()
        if not ok:
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            ok, frame = self.cap.read()
            if not ok:
                raise RuntimeError("Overlay video read failed even after rewind.")
        return frame

    def release(self):
        self.cap.release()

    def composite_screen(self, base_bgr: np.ndarray, overlay_bgr: np.ndarray) -> np.ndarray:
        if overlay_bgr.shape[:2] != base_bgr.shape[:2]:
            overlay_bgr = cv2.resize(
                overlay_bgr,
                (base_bgr.shape[1], base_bgr.shape[0]),
                interpolation=cv2.INTER_LINEAR
            )

        gray = cv2.cvtColor(overlay_bgr, cv2.COLOR_BGR2GRAY).astype(np.float32)
        mask = gray / 255.0
        # near-black suppression
        th = self.threshold / 255.0
        mask = np.clip((mask - th) / (1.0 - th), 0.0, 1.0)
        mask = (mask * self.alpha).astype(np.float32)

        base = base_bgr.astype(np.float32) / 255.0
        ov = overlay_bgr.astype(np.float32) / 255.0

        # screen blend
        screen = 1.0 - (1.0 - base) * (1.0 - ov)

        mask3 = np.repeat(mask[:, :, None], 3, axis=2)
        out = base * (1.0 - mask3) + screen * mask3
        out = np.clip(out * 255.0, 0, 255).astype(np.uint8)
        return out


def run_video_colab(
    input_video: str,
    overlay_video: str,
    out_video: str = "out_terminator.mp4",
    model_name: str = "yolov8n.pt",
    target_class: str = "person",
    conf: float = 0.25,
    iou: float = 0.7,
    imgsz: int = 640,
    overlay_alpha: float = 0.85,
    overlay_threshold: int = 10,
    add_edges: bool = True,
    add_scanlines: bool = True,
    max_frames: int = None,
):
    model = YOLO(model_name)

    cap = cv2.VideoCapture(input_video)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open input video: {input_video}")

    fps_in = cap.get(cv2.CAP_PROP_FPS)
    if not fps_in or fps_in <= 0:
        fps_in = 30.0

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(out_video, fourcc, fps_in, (w, h))

    overlay = OverlayPlayer(overlay_video, alpha=overlay_alpha, threshold=overlay_threshold)

    frame_idx = 0
    fps_est = 0.0
    prev_t = time.perf_counter()
    t0 = prev_t

    tc = target_class.strip().lower()

    try:
        while True:
            ok, frame = cap.read()
            if not ok:
                break

            now = time.perf_counter()
            dt = now - prev_t
            prev_t = now
            if dt > 0:
                fps_est = 0.9 * fps_est + 0.1 * (1.0 / dt) if fps_est > 0 else (1.0 / dt)

            # YOLO inference
            res = model.predict(source=frame, conf=conf, iou=iou, imgsz=imgsz, verbose=False)[0]

            dets = []
            if res.boxes is not None and len(res.boxes) > 0:
                xyxy = res.boxes.xyxy.cpu().numpy()
                confs = res.boxes.conf.cpu().numpy()
                clss = res.boxes.cls.cpu().numpy().astype(int)
                names = res.names

                for (x1, y1, x2, y2), c, cid in zip(xyxy, confs, clss):
                    cls_name = names[cid] if isinstance(names, (list, dict)) else str(cid)
                    dets.append({
                        "xyxy": (int(x1), int(y1), int(x2), int(y2)),
                        "conf": float(c),
                        "cls_id": int(cid),
                        "cls_name": str(cls_name),
                    })

            # pick target
            candidates = [d for d in dets if d["cls_name"].lower() == tc]
            target = None
            if candidates:
                def score(d):
                    x1, y1, x2, y2 = d["xyxy"]
                    area = max(0, x2 - x1) * max(0, y2 - y1)
                    return area * d["conf"]
                target = max(candidates, key=score)
            elif dets:
                target = max(dets, key=lambda d: d["conf"])

            # effect + our HUD
            stylized = apply_terminator_grade(
                frame, now - t0,
                add_edges=add_edges,
                add_scanlines=add_scanlines
            )
            hud = draw_hud(stylized, dets, target, fps_est, frame_idx)

            # overlay video
            ov = overlay.read()
            hud = overlay.composite_screen(hud, ov)

            writer.write(hud)

            frame_idx += 1
            if max_frames is not None and frame_idx >= max_frames:
                break

    finally:
        cap.release()
        writer.release()
        overlay.release()

    return out_video


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# ここを自分のファイル名に合わせて変更してください
INPUT_VIDEO = "sample_small.mp4"        # ①のアップロード動画
OVERLAY_VIDEO = "overlay1.mp4"   # ②のアップロード動画（t800-visionのoverlay）

out_path = run_video_colab(
    input_video=INPUT_VIDEO,
    overlay_video=OVERLAY_VIDEO,
    out_video="out_terminator.mp4",
    model_name="yolov8n.pt",     # 軽めでColab向き
    target_class="person",       # TERMINATE対象（例: person / car）
    conf=0.25,
    iou=0.7,
    imgsz=640,
    overlay_alpha=0.85,
    overlay_threshold=10,
    add_edges=True,
    add_scanlines=True,
)
out_path


'out_terminator.mp4'

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open("out_terminator.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="800" controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")


In [ ]:
OVERLAY_AUDIO = "overlay1.mp3"  # ③でアップロードした音声ファイル名

# 映像(out_terminator.mp4)に音声を付与して out_terminator_with_audio.mp4 を作る
# -shortest で短い方に合わせて終わります
!ffmpeg -y -i out_terminator.mp4 -i "{OVERLAY_AUDIO}" -c:v copy -c:a aac -shortest out_terminator_with_audio.mp4


In [ ]:
mp4 = open("out_terminator_with_audio.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="800" controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")


In [9]:
from google.colab import files
files.download("out_terminator.mp4")
# 音声版も作ったなら
files.download("out_terminator_with_audio.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>